# GeoSpatial Support

Azure SQL offer extensive GeoSpatial support. Thanks to this support you can easily create solution that can manipulate and take advantage of this to offer geospatial capabilities in your applications without having to integrate external libraries or solution, and without having to deal with all the huge complexity of planar mapping, spheric coordinates, projects and stuff like that.

- [Spatial Data](https://docs.microsoft.com/en-us/sql/relational-databases/spatial/spatial-data-sql-server?view=azuresqldb-current)
- [Spatial Data Types Overview](https://docs.microsoft.com/en-us/sql/relational-databases/spatial/spatial-data-types-overview?view=azuresqldb-current)


## Geometry or Geography?

 - Geometry: Planar data (flat-earth), Euclidean Coordinates (X,Y)
 - Geography: Ellipsoidal data (round-earth), GPS Coordinates (Lat, Long)



In [14]:
declare @p1 geometry = geometry::Parse('POINT(5 40)')
declare @p2 geometry = geometry::Parse('POINT(15 50)')

select 
    geometry::CollectionAggregate(g).ToString() as WKT,
    @p1.STDistance(@p2) as Distance,
    sqrt(power((@p1.STX-@p2.STX),2) + power((@p1.STY-@p2.STY),2)) as MyDistance -- Distance is calculated using the well known Pythagorean theorem 
from 
    (values (@p1), (@p2)) as T(g)

(1 row affected)

Total execution time: 00:00:00.026

WKT,Distance,MyDistance
"GEOMETRYCOLLECTION (POINT (5 40), POINT (15 50))",14.142135623730951,14.142135623730951


## Left Hand Rule for Polygons on Geography

Polygons must be created by using the "Left Hand Rule": if walking on the perimeter of the polygon, you're hand will be *inside* the polygon.

In [17]:
-- Note: Run on SSMS to see result

-- This is Microsoft Bulding 44
declare @bldg44 geography = geography::STPolyFromText('POLYGON((-122.13325278570375 47.64017318297519,-122.13287316603494 47.64017318297519,-122.13287803277747 47.64062243903447,-122.13261521918578 47.64062899746915,-122.13261521918578 47.64099954879444,-122.13333065653097 47.64099954879444,-122.13332578946104 47.641406168521314,-122.13376381233213 47.64139305179111,-122.13374434470715 47.640953639949714,-122.13400715862628 47.640947081555765,-122.13396822304897 47.6405732506378,-122.13327712039866 47.64057652985835,-122.13325278570375 47.64017318297519))', 4326)
select @bldg44

-- This is NOT Microsoft Building 44 :)
declare @not_bldg44 geography = geography::STPolyFromText('POLYGON((-122.13372461955305 47.64139305179111,-122.13333039885292 47.64139305179111,-122.13332553178299 47.64096675679025,-122.13266363057011 47.64096675679025,-122.13266849764004 47.64061588059897,-122.13286804128695 47.64061915981682,-122.13286804128695 47.640179741466255,-122.13325739509567 47.64017646222081,-122.13325739509567 47.640586367518694,-122.13402150150324 47.64058964673842,-122.13401663476071 47.640937243908155,-122.13372461940418 47.640933964710285,-122.13372461955305 47.64139305179111))', 4326)
select @not_bldg44


(1 row affected)

Total execution time: 00:00:00.042

(No column name)
0xE610000001040D000000CC53E231F1D14740B42CB13687885EC0CC53E231F1D14740B5A873FE80885EC0505E84EAFFD14740B548DD1281885EC0E87F882100D24740B5568BC47C885EC05469F1450CD24740B5568BC47C885EC05469F1450CD24740B4564E7D88885EC038A0EA9819D24740B55CE46888885EC088A9E22A19D24740B5D617968F885EC008EBD4C40AD24740B3A270448F885EC0D4DFD08D0AD24740B4EEC29293885EC04417E54DFED14740B52C74EF92885EC0D4296769FED14740B55AC29C87885EC0CC53E231F1D14740B42CB13687885EC001000000020000000001000000FFFFFFFF0000000003


## Playground

In [ ]:
drop table if exists #t;
create table #t (p varchar(100) primary key, g geometry);
go

-- Hollow Square
declare @p geometry = geometry::Parse('POLYGON((50 0, 150 0, 150 100, 50 100, 50 0), (80 30, 120 30, 120 70, 80 70, 80 30))')
select @p;
insert into #t values ('Hollow Square', @p)
go

-- Donut
declare @p geometry = geometry::Parse('POINT(50 50)').STBuffer(50)
declare @p2 geometry = geometry::Parse('POINT(50 50)').STBuffer(30)
declare @donut geometry =  @p.STDifference(@p2)
select @donut, @donut.ToString()
insert into #t values ('Donut', @donut)
go

-- Playground
select * from #t

declare @hs geometry = (select g from #t where p = 'Hollow Square')
select @hs.STArea() as Area

declare @hs geometry = (select g from #t where p = 'Hollow Square')
declare @d geometry = (select g from #t where p = 'Donut')
select @hs.STIntersects(@d)
select @hs.STIntersection(@d)

declare @p geometry = geometry::Parse('POINT(63 43)')
select * from #t where @p.STWithin(g) = 1

declare @p geometry = geometry::Parse('POINT(63 43)')
select * from #t where g.STContains(@p) = 1

declare @pt geometry = geometry::Parse('POINT(0 0)')
select p as Polygon, @pt.STDistance(g) as DistanceFrom from [#t]

## Store real-world data

Create a table that allows you to start geospatial data, using the [geography](https://docs.microsoft.com/en-us/sql/relational-databases/spatial/create-construct-and-query-geography-instances?view
) data type.

In [2]:
DROP TABLE IF EXISTS [dbo].[MetroBusData]
GO

CREATE TABLE [dbo].[MetroBusData]
(
	[Id] [int] NOT NULL,
	[LineDirectionId] [int] NOT NULL,
	[VehicleId] [int] NOT NULL,
	[TimestampUTC] [datetime2](7) NOT NULL,
	[TimestampLocal] [datetimeoffset](7) NOT NULL,
	[Location] [geography] NOT NULL,
	[Signage] [varchar](100) NOT NULL
)
GO

ALTER TABLE dbo.MetroBusData
ADD CONSTRAINT pk__MetroBusData PRIMARY KEY CLUSTERED ([Id]) 
WITH (DATA_COMPRESSION = PAGE)
GO

CREATE SPATIAL INDEX [isp] ON dbo.MetroBusData ([Location])
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.085

In this repo sample spatial data is provided in the file `busdata-20200901.bcp`. 

> Note: Transit scheduling, geographic, and real-time data provided by permission of King County
[King County Metro Developer Resources](https://kingcounty.gov/depts/transportation/metro/travel-options/bus/app-center/developer-resources.aspx)

You can import using the `bcp` tool:

```
bcp dbo.MetroBusData in C:\<your-folder>\busdata-20200901.bcp -S <azure-sql-server>.database.windows.net -U <user> -P <password> -n -d <database>
```

*or* you can use `BULK INSERT`. In such case you must copy the `busdata-20200901.bcp` file into an Azure Blob Account and the configure Azure SQL to be able to access that resource, like in the following code:

In [ ]:
/*
	Create the Database Master Key, if needed
*/
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'My-L0ng&Str0ng_P4ss0wrd!';
GO

/*
	Create database scoped credentials to store the Shared Access Signature (SAS) 
	needed to access the Azure Blob Container. More info on Azure Blob SAS here:
	
	https://docs.microsoft.com/en-us/azure/storage/storage-dotnet-shared-access-signature-part-1
	
    SAS can be created right from Azure Portal or Azure CLI or, the easiest way, using Azure Storage Explorer
*/

CREATE DATABASE SCOPED CREDENTIAL [azurestore-credentials]
WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
SECRET = 'sv=2019-02-02&......';
GO

/*
	Now create the external data source and point it to the Azure Blob Store
*/
CREATE EXTERNAL DATA SOURCE [azurestore]
WITH 
( 
	TYPE = BLOB_STORAGE,
 	LOCATION = 'https://<myaccount>.blob.core.windows.net',
 	CREDENTIAL= [azurestore-credentials]
);
GO

/*
Load Data
*/
BULK INSERT dbo.MetroBusData FROM '<container>/busdata-20200901.bcp' WITH (TABLOCK, DATAFILETYPE  = 'native', DATA_SOURCE = 'azurestore'); 


You can return a [WKT (Well-Known Text) representation](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) of the spatial data stored in the table. The following query is aggregating into a [MULTIPOINT](https://docs.microsoft.com/en-us/sql/relational-databases/spatial/multipoint) instance all the recorded location of bus line 221 going to Redmond Transit Center on the 1st Sept 2020.

In [3]:
with cte as
(
	select top (150)
		*
	from
		dbo.[MetroBusData]
	where
		[VehicleId] = 7335
	and 
		[Signage] = '221 REDMOND TC'
	and
		cast([TimestampLocal] as date) = '2020-09-01'
	order by
		[TimestampLocal] 
)
select 
	geography::UnionAggregate([Location]).ToString() 
from 
	cte
go

(1 row affected)

Total execution time: 00:00:01.814

(No column name)
"MULTIPOINT ((-122.125259 47.676662), (-122.132168 47.673602), (-122.129778 47.674289), (-122.129341 47.674423), (-122.133368 47.671363), (-122.13886 47.667814), (-122.143052 47.665949), (-122.133376 47.668819), (-122.143061 47.665518), (-122.134992 47.66613), (-122.143116 47.663072), (-122.143184 47.658569), (-122.143186 47.658228), (-122.143216 47.653378), (-122.142914 47.646951), (-122.142878 47.643758), (-122.142894 47.641544), (-122.142916 47.639636), (-122.142919 47.639005), (-122.143009 47.632834), (-122.143076 47.628921), (-122.143084 47.628354), (-122.143011 47.626358), (-122.143097 47.622232), (-122.143052 47.618655), (-122.14272 47.617038), (-122.133366 47.617007), (-122.13298 47.617004), (-122.126949 47.616982), (-122.121607 47.615356), (-122.121593 47.615113), (-122.121306 47.610201), (-122.1213 47.608226), (-122.121338 47.604443), (-122.121417 47.60263), (-122.120956 47.600603), (-122.146459 47.590857), (-122.146203 47.590201), (-122.120316 47.598139), (-122.143163 47.590852), (-122.146704 47.589024), (-122.140425 47.590864), (-122.135059 47.590809), (-122.148262 47.58647), (-122.148284 47.585663), (-122.147432 47.585301), (-122.132484 47.589164), (-122.143734 47.585381), (-122.143553 47.585412), (-122.144477 47.585078), (-122.127537 47.589008), (-122.115732 47.592079), (-122.152856 47.580437), (-122.1534 47.58019), (-122.152059 47.580434), (-122.121319 47.588977), (-122.121253 47.588977), (-122.142302 47.581775), (-122.141688 47.581244), (-122.143732 47.579707))"


As Azure SQL adheres to the Open Geospatial Consortium (OGS) standard, you can take the returned WKT result and copy and paste it in any solution compliant with OGS standard, for example [OpenLayers](https://openlayers.org/):

https://clydedacruz.github.io/openstreetmap-wkt-playground/

![Openlayers Playground](https://raw.githubusercontent.com/yorek/azure-sql-db-samples/master/samples/05-spatial/openlayers-playground.png)

You can create a polygon using the aformentioned website, for example:

```
POLYGON((-122.14357282700348 47.616901066671886,-122.141025341366 47.61685232450776,-122.14101421569923 47.617249758593886,-122.14283305463597 47.61725350816795,-122.14283861681452 47.61845704045888,-122.14351164303936 47.6184795362212,-122.14357282700348 47.616901066671886))
```

and you can check if any bus was within that polygon anytime between 17:00 and 17:30

In [4]:
declare @gf as geography = geography::STGeomFromText(
	'POLYGON((-122.14357282700348 47.616901066671886,-122.141025341366 47.61685232450776,-122.14101421569923 47.617249758593886,-122.14283305463597 47.61725350816795,-122.14283861681452 47.61845704045888,-122.14351164303936 47.6184795362212,-122.14357282700348 47.616901066671886))',
	4326
);
select 
	*
from
	dbo.[MetroBusData]
where
	[Signage] = '221 EDUCATION HILL'
and
	[TimestampLocal] BETWEEN '2020-09-01 17:00:00 -07:00' AND '2020-09-01 17:30:00 -07:00'
and
	[Location].STWithin(@gf) = 1
ORDER by
	[TimestampLocal] 
GO

(2 rows affected)

Total execution time: 00:00:00.084

Id,LineDirectionId,VehicleId,TimestampUTC,TimestampLocal,Location,Signage
2253549,2212,7301,2020-09-02 00:21:45.3766667,2020-09-01 17:21:45.3766667 -07:00,0xE6100000010CD331E719FBCE47401A69A9BC1D895EC0,221 EDUCATION HILL
2253550,2212,7301,2020-09-02 00:22:00.4200000,2020-09-01 17:22:00.4200000 -07:00,0xE6100000010CD331E719FBCE47401A69A9BC1D895EC0,221 EDUCATION HILL


You can also create collections of shapes, so that, for example, you can have the defined polygon and the points withing that polygon all togheter int he same shape

In [5]:

declare @gf as geography = geography::STGeomFromText(
	'POLYGON((-122.14357282700348 47.616901066671886,-122.141025341366 47.61685232450776,-122.14101421569923 47.617249758593886,-122.14283305463597 47.61725350816795,-122.14283861681452 47.61845704045888,-122.14351164303936 47.6184795362212,-122.14357282700348 47.616901066671886))',
	4326
);
with cte as
(
	select top (100)
		*
	from
		dbo.[MetroBusData]
	where
		[Signage] = '221 EDUCATION HILL'
	and
		[TimestampLocal] BETWEEN '2020-09-01 17:00:00 -07:00' AND '2020-09-01 17:30:00 -07:00'
	and
		[Location].STWithin(@gf) = 1
	order by
		Id desc
)
select 
	geography::STGeomCollFromText('GEOMETRYCOLLECTION(' + geography::UnionAggregate([Location]).ToString() + ', ' + @gf.ToString() + ')', 4326).ToString() 
from 
	cte
go

(1 row affected)

Total execution time: 00:00:00.039

(No column name)
"GEOMETRYCOLLECTION (POINT (-122.14244 47.617038), POLYGON ((-122.14357282700348 47.616901066671886, -122.141025341366 47.616852324507761, -122.14101421569923 47.617249758593886, -122.14283305463597 47.617253508167948, -122.14283861681452 47.618457040458878, -122.14351164303936 47.6184795362212, -122.14357282700348 47.616901066671886)))"
